# First steps in PySpark 

In this notebook we will learn the fundamentals of functional programming, as well as the basic abstraction of a distributed object in Spark, the RDD. The notebook has been divided into two parts:

Part 1: map/reduce basics

Part 2: Work with RDD and Pair RDD abstractions 

<a href = "http://yogen.io"><img src="http://yogen.io/assets/logo.svg" alt="yogen" style="width: 200px; float: right;"/></a>

# Part 1: map/reduce basics

![Hadoop Logo](https://upload.wikimedia.org/wikipedia/commons/thumb/0/0e/Hadoop_logo.svg/220px-Hadoop_logo.svg.png)
# **Apache Hadoop (MapReduce)**

It is an open source software framework written in Java for distributed storage and distributed processing of very large data sets on computer clusters built from commodity hardware. All the modules in Hadoop are designed with a fundamental assumption that hardware failures (of individual machines, or racks of machines) are common and thus should be automatically handled in software by the framework.

The core of Apache Hadoop consists of a storage part (Hadoop Distributed File System (HDFS)) and a processing part (MapReduce). Hadoop splits files into large blocks and distributes them amongst the nodes in the cluster. To process the data, Hadoop MapReduce transfers packaged code for nodes to process in parallel, based on the data each node needs to process. This approach takes advantage of data locality — nodes manipulating the data that they have on hand — to allow the data to be processed faster and more efficiently than it would be in a more conventional supercomputer architecture that relies on a parallel file system where computation and data are connected via high-speed networking.

![Map/shuffle/reduce](https://image.slidesharecdn.com/datasciencewebinar061312-120618165932-phpapp02/95/how-salesforcecom-uses-hadoop-41-728.jpg?cb=1340920667)

Since data and computation are distributed, we should avoid the use of variables, i.e. mutable data. Thus, in contrast to impertaive programming, we shall use the functional approach (lambda calculus).

### The goal of the following excercises is to understand basic lambda calculus with python.

### (1a) Functional programming in Python

So, what is Functional Programming? From Wikipedia: 

« …a  programing paradigm that treats computation as the evaluation of mathematical functions and **avoids changing-state and mutable  data**.»

It´s based upon Lambda calculus, wich consist of:
 * Function definition (declaration of expressions)
 * Function application (evaluation of those expressions)
 * Recursion (iteration)

We have already used this in python!!! :)

Recall the typical "lambda x: x+1" we have been using as the first argument of map, reduce and filter methods:
 * **map** maps each value in the input collection to a different value. It´s just the classical mathematical funciton we are used to!
 * **reduce** takes two values from the input collection and returns a new value (of the same type) by appliying a commutative operation to them. 
 * **filter** filters the elements in the input collection according to a certain (boolean) criteria.
 

**Mapping**

`map` is a Higher Order Function (HOF) that takes a function f and a sequence and returns a new sequence formed by applying f to each element in the original sequence. 

![map](https://cosminpupaza.files.wordpress.com/2015/10/map.png?w=505)

Very often we write lambdas, anonymous functions, to use in defining simple transformations with `map`:


In [1]:
input_list = [2,5,12]
list(map(lambda n: n**2, input_list))

[4, 25, 144]

In [2]:
list(map(lambda s: s+'s', ['moto','coche']))

['motos', 'coches']

We can also use named functions, of course. Note that we pass them as arguments to `map`, not execute them. You can think of them as tools we give to `map` for it to use.

In [4]:
def plural(string):
    return string+'s'
plural('moto')

'motos'

In [5]:
list(map(plural, ['moto','coche']))

['motos', 'coches']

Consider the following very common pattern: create empty list, append transformed elements one by one:

In [6]:
input_list = ['moto','coche']
plurals = []

for word in input_list:
    plurals.append(plural(word))
    
plurals

['motos', 'coches']

We can abstract over every possible list by defining a function that takes the list as an argument:

In [10]:
def pluralize_list(input_list):
    plurals=[]
    
    for word in input_list:
        plurals.append(plural(word))
        
    return plurals

pluralize_list(['manzana','pera'])

['manzanas', 'peras']

Consider this other piece of code: we are doing a very similar thing, but with a different function. A lot of the code is common between the two little programs.

Wouldn't it be convenient if we could abstract over every function, as we did over every list?

In [11]:
def initial(word):
    return word[0]

input_list = ['moto','coche']
initials=[]

for word in input_list:
    initials.append(initial(word))
    
initials

['m', 'c']

We can, taking the function to be applied as an argument. This is because Python treats functions as first-class objects, so they can be used as arguments to other functions.

In [12]:
def process_list(function, input_list):
    result=[]
    
    for element in input_list:
        result.append(function(element))
    return result

In [13]:
fruits = ['manzana','pera']
process_list(plural,fruits)

['manzanas', 'peras']

In [14]:
process_list(initial,fruits)

['m', 'p']

That function we just wrote, `process_list`, is exactly what `map` is

In [15]:
list(map(plural, fruits))

['manzanas', 'peras']

In [16]:
list(map(initial, fruits))

['m', 'p']

**Filtering**

`filter` encodes another very common pattern: building a sequence from some of the elements of an input sequence, deciding whether to include each based on the result of evaluating a function, often called the `predicate`, on each of the elements.


![filter](https://cosminpupaza.files.wordpress.com/2015/11/filter.png?w=405)

In [17]:
input_list = [2,5,12]
list(filter(lambda n: n<10, input_list))

[2, 5]

Again, we can use either lambdas or named functions.

In [18]:
def predicate(n):
    return n<10

res = []
for element in input_list:
    if predicate(element):
        res.append(element)
        
res

[2, 5]

In [19]:
list(filter(predicate, input_list))

[2, 5]

#### Exercise

write a filter to extract those words that have more than 3 letters from `collection_of_strings`

In [21]:
collection_of_strings = ['Holi','que','tal','te','van','esas','clases','sin','ventanas']

In [22]:
list( filter(lambda word: len(word)>3, collection_of_strings))

['Holi', 'esas', 'clases', 'ventanas']

#### Exercise

Write a filter to extract prime numbers from the following list of integers:

```python
[12, 17, 19, 18, 23, 24]
```

You will need to write a function that determines whether a single number is a prime or not.

In [23]:
def isprime(num):
    for i in range(2,num):
        if (num % i) == 0:
            return False
    return True
list_of_numbers = [12, 17, 19, 18, 23, 24]
list( filter(isprime,list_of_numbers))

[17, 19, 23]

`map` and `filter` are not  used explicitly very often in Python, in part because of their cumbersome syntax. 

Nevertheless, there is a well-known feature in Python that is in fact just a convenient way to write `map`s and `filter`s: **list comprehensions**:

In [25]:
[number for number in list_of_numbers if isprime(number)]

[17, 19, 23]

In [27]:
[number**2 for number in list_of_numbers]

[144, 289, 361, 324, 529, 576]

In [28]:
[number**2 for number in list_of_numbers if isprime(number)]

[289, 361, 529]

The above list comprehension is exactly equivalent to:

In [29]:
list( map(lambda n: n**2, filter(isprime,list_of_numbers)) )

[289, 361, 529]

Which is a pipeline of two steps: first a `filter`, then a `map`

**Reducing** 

`reduce` is the third basic foundation of functional programming. Reduce uses a function, often called the `combiner`, to transform a sequence of elements of type T (that is, any type) into a single T. The combiner must take 2 Ts and return only one T.


Recall it must be commutative! Think about the importance of this when parallelizing computations

![reduce](https://cosminpupaza.files.wordpress.com/2015/11/reduce.png?w=500)

In [30]:
from functools import reduce

def add_2_elements(a,b):
    return a + b

reduce(add_2_elements, [9,2,4,7,9,5,6,4,3,3])

52

In [34]:
from functools import reduce

def substract_2_elements(a,b):
    return a - b

reduce(substract_2_elements, [10,9,8])

-7

#### Exercise

Write a function_x such that, when applied with `reduce`, that reduce returns the highest number in a list. Don't use the built-in `max`!

In [35]:
def function_x(a,b):
    return (a if a>b else b)
reduce(function_x,[9,2,4,7,3,1,8])

9

## (1b) Exercise: Calculate the mean of a collection of real numbers using map/reduce
Recall:

$$\bar x = \frac{\sum_{i=1}^{N} x_i}{N} $$

It´s straightforward to do this with python built-in mehots sum() and len(). However, how would you do that with map/reduce? We have already shown how to sum the elements of an array. Thus, you have to calculate the length of the array. For this:
 * Create another array of the same size, consisting of 1s.
 * Sum the elements of that array

#### First part

* Do a reduce to do the sum, and a different map-reduce to get the length

In [36]:
list_of_numbers = [9,2,4,7,9,5,6,4,3,3]
reduce(lambda a,b: a+b, map(lambda n: 1,list_of_numbers))

10

That was also a pipeline of two steps. It's equivalent to:

In [37]:
result_of_first_step = map(lambda n: 1,list_of_numbers)
length = reduce(lambda a,b: a+b, result_of_first_step)

In [38]:
length

10

In [40]:
summation = reduce(lambda a,b: a+b, list_of_numbers)
summation

52

#### Combine them in one pass

Think about what it would mean to finish one computation and only then do another in a distributed environment.

Combine both in one pass:

In [41]:
def f(a):
    return (a,1)

result_of_first_step = list(map(f, list_of_numbers))
result_of_first_step

[(9, 1),
 (2, 1),
 (4, 1),
 (7, 1),
 (9, 1),
 (5, 1),
 (6, 1),
 (4, 1),
 (3, 1),
 (3, 1)]

In [42]:
def g(tuple_1,tuple_2):
    running_total_1, running_count_1 = tuple_1
    running_total_2, running_count_2 = tuple_2
    
    running_total = running_total_1 + running_total_2
    running_count = running_count_1 + running_count_2
    
    return (running_total, running_count)

reduce(g,result_of_first_step)

(52, 10)

In [47]:
def g(tuple_1,tuple_2):
    
    return (tuple_1[0] + tuple_2[0], tuple_1[1] + tuple_2[1])

reduce(g,result_of_first_step)

(52, 10)

In [48]:
sum_, count = reduce(lambda t1,t2: (t1[0]+t2[0],t1[1]+t2[1]) ,result_of_first_step)

In [43]:
sum_, count = reduce(g,result_of_first_step)

In [49]:
mean = sum_/count
mean

5.2

## (1c) Exercise: Calculate the standard deviation of a collection of real numbers

We'll use the typical definition of the standard deviation:

$$\sigma_x^2 = \frac{\sum_{i=1}^{N} (x_i-\bar x)^2}{N-1}$$

For this, use the *mean* and *count* variables from the previous excercise.

In [50]:
list_of_numbers

[9, 2, 4, 7, 9, 5, 6, 4, 3, 3]

In standard idiomatic Python we'd do it like this:

With `map` and `reduce`:

In [52]:
square_differences=map(lambda n: (n-mean)**2,list_of_numbers)
reduce(lambda a,b: a+b, square_differences)/(count-1)

6.177777777777778

It is not correct to include the count in the reduce operation!

In [53]:
square_differences=map(lambda n: (n-mean)**2,list_of_numbers)
reduce(lambda a,b: (a+b)/(count-1), square_differences)

0.5996323423152771

#### Calculate the standard deviation in one pass.

Again, think about what it would mean to finish one computation and only then do another in a distributed environment.


If we can, we should avoid broadcasting variables all over the cluster. In this case, the following alternative definition of standard deviation can come in handy:

$$\sigma_x^2 = \frac{\sum_{i=1}^{N} (x_i-\bar x)^2}{N-1} =
\frac{\sum_{i=1}^{N} (x_i^2+{\bar x}^2-2x_i\bar x)}{N-1} =
\frac{1}{N-1}\left(\sum_i x_i^2-N\bar x^2\right)$$



In [54]:
list_of_numbers

[9, 2, 4, 7, 9, 5, 6, 4, 3, 3]

In [55]:
f = lambda n:(n**2,n,1)
f(3)

(9, 3, 1)

In [58]:
def f(n):
    return (n**2,n,1)

def g(tuple_1,tuple_2):
    
    partial_sum_sq = tuple_1[0]+tuple_2[0]
    partial_sum_   = tuple_1[1]+tuple_2[1]
    partial_n      = tuple_1[2]+tuple_2[2]
    
    return (partial_sum_sq, partial_sum_, partial_n)

sum_sq, sum_, n = reduce(g,map(f,list_of_numbers))
variance = (sum_sq-n*((sum_/n)**2))/(n-1)
variance = 1/(n-1)*(sum_sq-n*(sum_/n)**2)

In [59]:
variance

6.177777777777774

#### Side note

In Python 2, map and filter returned lists. In Python 3, they return generators, which are lazy collections. They are somewhat similar to files in that they can be depleted of elements after iterating through them.

## (1d) Twe 'word-count' problem: creating histograms
Given a set of keys in an input collection, calculate the frequency of each key. 

In order to understand better how map/reduce works, we will implement this simple calculation in several forms.

We are going to use as test case a list of elements chosen from a small set:

In [65]:
import random

random.seed(42)
list_of_cakes = [random.choice(['sacher',
                               'santiago',
                               'san marcos',
                               'coca']) 
                 for _ in range(100)]

### (1d.1) Simple approach

 * Start with an empty dict
 * If a new key is not present in the dict, create it.
 * Otherwise, increase the frequency of the key by one.

In [66]:
count_of_cakes ={}

for cake in list_of_cakes:
    if cake not in count_of_cakes:
        count_of_cakes[cake] = 1
    else:
        count_of_cakes[cake] += 1
        
count_of_cakes

{'sacher': 27, 'san marcos': 24, 'santiago': 29, 'coca': 20}

### (1d.2) Map/reduce

* Recall that *reduce* applies an operation to 2 elements of the same type, and returns another element of that type. Thus, the first thing to do is to map our collection to the type of the output. 
 
* The final result will be a dictionary of words in the vocabulary and counts. Therefore, we need to map each input word to this type.
 
* Then, we have to define a function, the combiner, that combines these dictionaries two at a time.

In [69]:
from collections import Counter

def combiner(dict_a, dict_b):
    result = Counter(dict_a)+Counter(dict_b)
    return dict(result)

In [70]:
##Counter(list_of_cakes)

Counter({'sacher': 27, 'san marcos': 24, 'santiago': 29, 'coca': 20})

In [71]:
reduce(combiner,map(lambda cake: {cake: 1},list_of_cakes))

{'sacher': 27, 'san marcos': 24, 'santiago': 29, 'coca': 20}

# Part 2: Spark. Work with RDD and Pair RDD abstractions 

![drawing](https://prateekvjoshi.files.wordpress.com/2015/10/1-main4.png)

# ** Apache Spark**

Apache Spark is an open source cluster computing framework originally developed in the AMPLab at University of California, Berkeley but was later donated to the Apache Software Foundation where it remains today. In contrast to Hadoop's two-stage disk-based MapReduce paradigm, Spark's multi-stage in-memory primitives provides performance up to 100 times faster for certain applications.

![](http://image.slidesharecdn.com/sparkandshark-120620130508-phpapp01/95/spark-and-shark-8-728.jpg?cb=1340197567)

By allowing user programs to load data into a cluster's memory and query it repeatedly, Spark is well-suited to machine learning algorithms.
![](http://spark.apache.org/images/logistic-regression.png)

Spark comes with a number of components that provide flexibility and generality.

<img src="http://spark.apache.org/images/spark-stack.png" alt="Drawing" style="width: 500px;"/>


## In this part, we keep on working on the word-count example, this time with spark. The basic abstraction of Spark is the Resilient Distributed Dataset (RDD):

#### «RDDs are fault-tolerant, parallel data structures that let users explicitly persist intermediate results in memory, control their partitioning to optimize data placement, and manipulate them using a rich set of operators.»

 * Read only, partitioned collection of records (immutable).
 * Stores the transformations used to build a dataset (its lineage), instead of the data itself. This property ensures fault-tolerance.
 * Users can control partitioning and persistence (caching).
 * RDDs are statically typed.
 * … and yes, everything is written in scala ;p. So you could use learning a bit of it!
 
<img src="http://eng.trueaccord.com/wp-content/uploads/2014/10/scala-logo.png" alt="Drawing" style="width: 200px;"/>

#### We will be trying to understand this abstraction with simple examples, using the [Python API](http://spark.apache.org/docs/latest/api/python/index.html)




### ** (2a) Create a base RDD: parallelize, actions and transformations **
We'll start by generating a base RDD by using a Python list and the `sc.parallelize` method.  Then we'll print out the type of the base RDD.

We use sc.parallelize to convert a standard Python collection into an RDD.

In [72]:
list_of_cakes

['sacher',
 'sacher',
 'san marcos',
 'santiago',
 'santiago',
 'santiago',
 'sacher',
 'sacher',
 'coca',
 'sacher',
 'sacher',
 'sacher',
 'santiago',
 'santiago',
 'sacher',
 'santiago',
 'coca',
 'santiago',
 'coca',
 'san marcos',
 'sacher',
 'santiago',
 'coca',
 'san marcos',
 'san marcos',
 'santiago',
 'santiago',
 'san marcos',
 'sacher',
 'sacher',
 'coca',
 'sacher',
 'san marcos',
 'san marcos',
 'san marcos',
 'sacher',
 'coca',
 'sacher',
 'coca',
 'sacher',
 'san marcos',
 'san marcos',
 'santiago',
 'sacher',
 'sacher',
 'santiago',
 'san marcos',
 'sacher',
 'santiago',
 'sacher',
 'coca',
 'san marcos',
 'coca',
 'san marcos',
 'santiago',
 'san marcos',
 'san marcos',
 'santiago',
 'san marcos',
 'sacher',
 'santiago',
 'santiago',
 'santiago',
 'coca',
 'coca',
 'san marcos',
 'santiago',
 'san marcos',
 'sacher',
 'santiago',
 'sacher',
 'san marcos',
 'coca',
 'san marcos',
 'sacher',
 'santiago',
 'san marcos',
 'santiago',
 'coca',
 'coca',
 'coca',
 'santiago'

In [73]:
type(list_of_cakes)

list

In [74]:
sc

<SparkContext master=local[*] appName=PySparkShell>

In [76]:
my_first_rdd=sc.parallelize(list_of_cakes)
type(my_first_rdd)

pyspark.rdd.RDD

In Spark, `map` is a method of the RDD type.

In [77]:
my_second_rdd = my_first_rdd.map(plural)

In [78]:
my_second_rdd

PythonRDD[2] at RDD at PythonRDD.scala:49

**Nothing has actually happened!**

`parallellize` tells spark to distribute the data, but this is not actually done until we perform some action.

Possible actions include counting, collecting, reducing, taking, etc. Take a look at the [Spark programming guide](http://spark.apache.org/docs/2.2.0/rdd-programming-guide.html#actions)


In [80]:
my_second_rdd.count()

100

#### Exercise

Calculate, from the rdd generated from list_of_cakes, how many cakes we have whose name does **not** start with an 's'

In [81]:
non_s_cakes = my_first_rdd.filter(lambda cake: cake[0] != 's')

In [85]:
non_s_cakes.take(3)

['coca', 'coca', 'coca']

In [87]:
non_s_cakes.collect()

['coca',
 'coca',
 'coca',
 'coca',
 'coca',
 'coca',
 'coca',
 'coca',
 'coca',
 'coca',
 'coca',
 'coca',
 'coca',
 'coca',
 'coca',
 'coca',
 'coca',
 'coca',
 'coca',
 'coca']

In [90]:
non_s_cakes.count()

20

In [88]:
non_s_cakes.map(lambda n:1).reduce(lambda a,b: a+b)

20

A very common way to write these chained operations in Spark is to put each on a line. Due to Python's syntax, we need to escape the newline character with a backslash.

In [93]:
my_first_rdd\
     .filter(lambda cake: cake[0]!='s')\
     .map(lambda n:1)\
     .reduce(lambda a,b: a+b)

20

In [94]:
count_cakes_non_s = my_first_rdd\
     .filter(lambda cake: cake[0]!='s')\
     .map(lambda n:1)\
     .reduce(lambda a,b: a+b)

### Actions and Transformations

There are two main types of operations in Spark:
[Actions](http://spark.apache.org/docs/2.2.0/rdd-programming-guide.html#actions) and [Transformations](http://spark.apache.org/docs/2.2.0/rdd-programming-guide.html#transformations).

**Transformations** produce an RDD. Some of the most important are `map`, `filter`, and `reduceByKey`. `sc.parallelize` and `sc.textFile` are not technically Transformations but you can think of them as such in one very important respect: they are *lazyly evaluated*. That is, when we perform a Transformation, we are only describing the operation to be performed at some point in the future. They attach a node to the execution graph.

**Actions** are implemented as methods on an RDD, and return an object of a type **that is not an RDD**. When we perform an Action, we give the order to execute the previously described transformations: we trigger the execution of the graph. Some of the most important are `reduce`, `collect`, `take`, `takeOrdered`, and `count`.

In [113]:
numbers2 = sc.parallelize(range(100000))

In [114]:
numbers2

PythonRDD[26] at RDD at PythonRDD.scala:49

In [115]:
import math
%time roots2 = numbers2.map(lambda x: math.sqrt(x))

CPU times: user 14 µs, sys: 0 ns, total: 14 µs
Wall time: 17.9 µs


In [116]:
%time collected_roots = roots2.count()

KeyboardInterrupt: 

### **(2b) Persisting and the RDD lineage**

So far, we have seen that Spark RDDs are *lazyly evaluated*, i.e. nothing is actually done until an action is performed. In the RDD, the set of transformations to be applied are remembered: this is known as its *lineage*. It has the important consequence of making Spark RDDs *fault tolerant* automatically.

![](http://images.slideplayer.com/14/4499833/slides/slide_10.jpg) 

It might be interesting to store some intermediate results, though: perhaps because we want to apply several different transformations starting from that point, or because we are going to apply an iterative computation (as is customary in machine learning algorithms). For this, Spark has [several ways of persisting](http://spark.apache.org/docs/latest/rdd-programming-guide.html#rdd-persistence)

In [117]:
from pyspark import StorageLevel
StorageLevel.MEMORY_AND_DISK

StorageLevel(True, True, False, False, 1)

In [121]:
StorageLevel.MEMORY_ONLY

StorageLevel(False, True, False, False, 1)

StorageLevel cannot be changed after it has been set. We need to `unpersist` first.

In [118]:
non_s_cakes.persist()

PythonRDD[5] at collect at <ipython-input-83-0ab577d39d8a>:1

In [119]:
non_s_cakes.getStorageLevel()

StorageLevel(False, True, False, False, 1)

In [122]:
non_s_cakes.persist(StorageLevel.MEMORY_AND_DISK_2)

Py4JJavaError: An error occurred while calling o70.persist.
: java.lang.UnsupportedOperationException: Cannot change storage level of an RDD after it was already assigned a level
	at org.apache.spark.rdd.RDD.persist(RDD.scala:170)
	at org.apache.spark.rdd.RDD.persist(RDD.scala:195)
	at org.apache.spark.api.java.JavaRDD.persist(JavaRDD.scala:47)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:357)
	at py4j.Gateway.invoke(Gateway.java:282)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.GatewayConnection.run(GatewayConnection.java:

In [123]:
import math
long_rdd = sc.parallelize(range(1000000000))\
           .map(math.sqrt)\
           .reduce(lambda a,b: a+b)

In [126]:
long_rdd

21081851051978.81

In [127]:
long_rdd_2=sc.parallelize(range(1000000000))\
        .map(math.sqrt).persist(StorageLevel.MEMORY_AND_DISK_2)

In [128]:
%time long_rdd_2.map(lambda a:-a).take(5)

CPU times: user 7.32 ms, sys: 3.18 ms, total: 10.5 ms
Wall time: 1min 2s


[-0.0, -1.0, -1.4142135623730951, -1.7320508075688772, -2.0]

In [129]:
%time long_rdd_2.reduce(lambda a,b: a+b)

CPU times: user 16.1 ms, sys: 9.34 ms, total: 25.4 ms
Wall time: 3min 59s


21081851051978.81

### **(2c) Partitioning **

One important parameter for parallel collections is the number of partitions to cut the dataset into. Spark will run one task for each partition of the cluster. Typically you want 2-4 partitions for each CPU in your cluster.

To get the number of partitions of an RDD, just use `getNumPartitions()` on your RDD. You can change the partitions during RDD creation (with `parallelize(collection,numPartitions)` or `fromTextFile(file,numPartitions)`), or afterwards with methods like `repartition(), coalesce()`, etc.

In [130]:
long_rdd_2.getNumPartitions()

4

In [135]:
l1 = [1,2]

In [144]:
short_rdd = sc.parallelize(l1)
short_rdd.getNumPartitions()

4

We can see the partitions using [glom()](http://spark.apache.org/docs/latest/api/python/pyspark.html?highlight=glom#pyspark.RDD.glom): it retruns an RDD created by coalescing all elements within each partition into a list.

In [145]:
short_rdd.glom().collect()

[[], [1], [], [2]]

In [139]:
num_partitions = 2

In [146]:
short_rdd = sc.parallelize(l1,num_partitions)

In [147]:
short_rdd.getNumPartitions()

2

In [148]:
short_rdd.glom().collect()

[[1], [2]]

In [149]:
medium_rdd = sc.parallelize(range(20))

In [150]:
medium_rdd.glom().collect()

[[0, 1, 2, 3, 4], [5, 6, 7, 8, 9], [10, 11, 12, 13, 14], [15, 16, 17, 18, 19]]

Partitions are one of the most powerfull concepts in Spark: you can decide how to distribute your data so it can fit in memory, and more importantly, you can perform computations on each partition *before* speaking to other partitions. This can have an enormous impact on performance

### **(2c) Pair RDDs: *grouping* strategies in Spark**

The next step in writing our word counting program is to create a new type of RDD, called a pair RDD. A pair RDD is an RDD where each element is a pair tuple (k, v) where k is the key and v is the value. In this example, we will create a pair consisting of ('<word>', 1) for each word element in the RDD, as we did in the map/reduce version of the histogram in Python, section (1d.2).

We can create the pair RDD using the map() transformation with a lambda() function to create a new RDD.

In [152]:
cake_rdd = sc.parallelize(list_of_cakes)
cake_rdd

ParallelCollectionRDD[49] at parallelize at PythonRDD.scala:184

In [159]:
pair_rdd = cake_rdd.map(lambda cake: (cake,1))
pair_rdd.collect()

[('sacher', 1),
 ('sacher', 1),
 ('san marcos', 1),
 ('santiago', 1),
 ('santiago', 1),
 ('santiago', 1),
 ('sacher', 1),
 ('sacher', 1),
 ('coca', 1),
 ('sacher', 1),
 ('sacher', 1),
 ('sacher', 1),
 ('santiago', 1),
 ('santiago', 1),
 ('sacher', 1),
 ('santiago', 1),
 ('coca', 1),
 ('santiago', 1),
 ('coca', 1),
 ('san marcos', 1),
 ('sacher', 1),
 ('santiago', 1),
 ('coca', 1),
 ('san marcos', 1),
 ('san marcos', 1),
 ('santiago', 1),
 ('santiago', 1),
 ('san marcos', 1),
 ('sacher', 1),
 ('sacher', 1),
 ('coca', 1),
 ('sacher', 1),
 ('san marcos', 1),
 ('san marcos', 1),
 ('san marcos', 1),
 ('sacher', 1),
 ('coca', 1),
 ('sacher', 1),
 ('coca', 1),
 ('sacher', 1),
 ('san marcos', 1),
 ('san marcos', 1),
 ('santiago', 1),
 ('sacher', 1),
 ('sacher', 1),
 ('santiago', 1),
 ('san marcos', 1),
 ('sacher', 1),
 ('santiago', 1),
 ('sacher', 1),
 ('coca', 1),
 ('san marcos', 1),
 ('coca', 1),
 ('san marcos', 1),
 ('santiago', 1),
 ('san marcos', 1),
 ('san marcos', 1),
 ('santiago', 1),


### ** (2c.1) `groupByKey()` approach **
An approach you might first consider (we'll see shortly that there are better ways) is based on using the [groupByKey()](http://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD.groupByKey) transformation. As the name implies, the `groupByKey()` transformation groups all the elements of the RDD with the same key into a single list in one of the partitions. There are two problems with using `groupByKey()`:
  + The operation requires a lot of data movement to move all the values into the appropriate partitions.
  + The lists can be very large. Consider a word count of English Wikipedia: the lists for common words (e.g., the, a, etc.) would be huge and could exhaust the available memory in a worker.
 
Use `groupByKey()` to generate a pair RDD of type `('word', iterator)`. Next, sum the iterator using a `map()` transformation.  The result should be a pair RDD consisting of (word, count) pairs.

In [160]:
grouped_rdd = pair_rdd.groupByKey()
grouped_rdd.collect()

[('sacher', <pyspark.resultiterable.ResultIterable at 0x11b6dcc88>),
 ('san marcos', <pyspark.resultiterable.ResultIterable at 0x11b6dc048>),
 ('santiago', <pyspark.resultiterable.ResultIterable at 0x11b6ea550>),
 ('coca', <pyspark.resultiterable.ResultIterable at 0x11b6eafd0>)]

In [161]:
res = grouped_rdd.map(lambda kv_tuple: (kv_tuple[0],len(kv_tuple[1])))

In [162]:
res.collect()

[('sacher', 27), ('san marcos', 24), ('santiago', 29), ('coca', 20)]

### ** (2c.2)  `reduceByKey` approach **
A better approach is to start from the pair RDD and then use the [reduceByKey()](http://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD.reduceByKey) transformation to create a new pair RDD. 

The `reduceByKey()` transformation gathers together pairs that have the same key and applies the function provided to two values at a time, iteratively reducing all of the values to a single value. `reduceByKey()` operates by applying the function first within each partition on a per-key basis and then across the partitions, allowing it to scale efficiently to large datasets.

![](https://databricks.gitbooks.io/databricks-spark-knowledge-base/content/images/reduce_by.png)

In [163]:
help(grouped_rdd.reduceByKey)

Help on method reduceByKey in module pyspark.rdd:

reduceByKey(func, numPartitions=None, partitionFunc=<function portable_hash at 0x10e02dea0>) method of pyspark.rdd.PipelinedRDD instance
    Merge the values for each key using an associative and commutative reduce function.
    
    This will also perform the merging locally on each mapper before
    sending results to a reducer, similarly to a "combiner" in MapReduce.
    
    Output will be partitioned with C{numPartitions} partitions, or
    the default parallelism level if C{numPartitions} is not specified.
    Default partitioner is hash-partition.
    
    >>> from operator import add
    >>> rdd = sc.parallelize([("a", 1), ("b", 1), ("a", 1)])
    >>> sorted(rdd.reduceByKey(add).collect())
    [('a', 2), ('b', 1)]



In [164]:
count_of_cakes = pair_rdd.reduceByKey(lambda value_1,value_2: value_1+value_2)
count_of_cakes.collect()

[('sacher', 27), ('san marcos', 24), ('santiago', 29), ('coca', 20)]

#### Summary: word count in Spark:

In [165]:
cake_rdd.map(lambda cake:(cake,1)).reduceByKey(lambda a,b: a+b).collect()

[('sacher', 27), ('san marcos', 24), ('santiago', 29), ('coca', 20)]

## (2d) Apply word count to a file

### ** (2d.1) Load a text file **
For the next part of this lab, we will use the [Complete Works of William Shakespeare](http://www.gutenberg.org/ebooks/100) from [Project Gutenberg](http://www.gutenberg.org/wiki/Main_Page). To convert a text file into an RDD, we use the `SparkContext.textFile()` method. We also apply the recently defined `removePunctuation()` function using a `map()` transformation to strip out the punctuation and change all text to lowercase.  Since the file is large we use `take(15)`, so that we only print(15 lines.)

In [166]:
!head shakespeare.txt

1609

THE SONNETS

by William Shakespeare



                     1
  From fairest creatures we desire increase,


In [168]:
shakespeare_lines = sc.textFile('shakespeare.txt')
shakespeare_lines

shakespeare.txt MapPartitionsRDD[83] at textFile at NativeMethodAccessorImpl.java:0

In [169]:
shakespeare_lines.take(5)

['1609', '', 'THE SONNETS', '', 'by William Shakespeare']

### ** (2d.2) Capitalization and punctuation **
Real world files are more complicated than the data we have been using in this lab. Some of the issues we have to address are:
  + Words should be counted independent of their capitialization (e.g., Spark and spark should be counted as the same word).
  + All punctuation should be removed.
  + Any leading or trailing spaces on a line should be removed.
 
Define the function `removePunctuation` that converts all text to lower case, removes any punctuation, and removes leading and trailing spaces.  Use the Python [re](https://docs.python.org/2/library/re.html) module to remove any text that is not a letter, number, or space. Reading `help(re.sub)` might be useful.

In [174]:
lower_cased = shakespeare_lines.map(lambda line: line.lower())
lower_cased.take(5)

['1609', '', 'the sonnets', '', 'by william shakespeare']

In [175]:
import re
help(re.sub)

Help on function sub in module re:

sub(pattern, repl, string, count=0, flags=0)
    Return the string obtained by replacing the leftmost
    non-overlapping occurrences of the pattern in string by the
    replacement repl.  repl can be either a string or a callable;
    if a string, backslash escapes in it are processed.  If it is
    a callable, it's passed the match object and must return
    a replacement string to be used.



In [176]:
clean = lower_cased.map(lambda line: re.sub('[^\w]','',line))
clean.take(5)

['1609', '', 'thesonnets', '', 'bywilliamshakespeare']

http://www.regular-expressions.info/quickstart.html

https://regex101.com/

### ** (2d.3) Words from lines **
Before we can use the `wordcount()` function, we have to address two issues with the format of the RDD:
  + The first issue is that  that we need to split each line by its spaces.
  + The second issue is we need to filter out empty lines.
 
Apply a transformation that will split each element of the RDD by its spaces. For each element of the RDD, you should apply Python's string [split()](https://docs.python.org/3/library/stdtypes.html#str.split) function. You might think that a `map()` transformation is the way to do this, but think about what the result of the `split()` function will be.

Check out the Spark Programming Guide for an alternative transformation that helps us here.

In [179]:
'     '.split(' ')

['', '', '', '', '', '']

In [180]:
'     '.split()

[]

In [183]:
'hola      caracola'.split(' ')

['hola', '', '', '', '', '', 'caracola']

In [212]:
'hola      caracola'.split()

['hola', 'caracola']

In [185]:
clean.map(str.split)
clean.map(lambda line:line.split(' ')).take(10)

[['1609'],
 [''],
 ['thesonnets'],
 [''],
 ['bywilliamshakespeare'],
 [''],
 [''],
 [''],
 ['1'],
 ['fromfairestcreatureswedesireincrease']]

In [208]:
words = shakespeare_lines.map(lambda line: line.lower())\
                 .map(lambda line: line.split())
                 #.map(lambda line: re.sub('[^\w]','',line))

In [209]:
words.take(10)

[['1609'],
 [],
 ['the', 'sonnets'],
 [],
 ['by', 'william', 'shakespeare'],
 [],
 [],
 [],
 ['1'],
 ['from', 'fairest', 'creatures', 'we', 'desire', 'increase,']]

In [210]:
words = shakespeare_lines.map(lambda line: line.lower())\
                 .flatMap(lambda line: line.split())

In [211]:
words.take(10)

['1609',
 'the',
 'sonnets',
 'by',
 'william',
 'shakespeare',
 '1',
 'from',
 'fairest',
 'creatures']

In [213]:
words

PythonRDD[101] at RDD at PythonRDD.scala:49

In [216]:
words.count()

883320

In [214]:
words2 = words.filter(lambda word: word !='')
words2.take(10)

['1609',
 'the',
 'sonnets',
 'by',
 'william',
 'shakespeare',
 '1',
 'from',
 'fairest',
 'creatures']

In [217]:
words2.count()

883320

### (2d.4) Count the words and show the top 15

We know the drill at this point, don't we? We map to a tuple then `reduceByKey`

We can view the top 15 words by using the `takeOrdered()` action; however, since the elements of the RDD are pair tuples, we need a custom sort function that sorts using the value part of the pair rather than the key.

You'll notice that many of the words are common English words (know as stopwords).

Use our map reduce and and `takeOrdered()` to obtain the fifteen most common words and their counts.

In [218]:
counts = words\
        .map(lambda word:(word,1))\
        .reduceByKey(lambda a,b: a+b)\
        .cache()

In [219]:
counts.take(5)

[('shakespeare', 38),
 ('1', 13),
 ('fairest', 39),
 ('creatures', 27),
 ('we', 3201)]

In [220]:
help(counts.takeOrdered)

Help on method takeOrdered in module pyspark.rdd:

takeOrdered(num, key=None) method of pyspark.rdd.PipelinedRDD instance
    Get the N elements from an RDD ordered in ascending order or as
    specified by the optional key function.
    
    .. note:: this method should only be used if the resulting array is expected
        to be small, as all the data is loaded into the driver's memory.
    
    >>> sc.parallelize([10, 1, 2, 9, 3, 4, 5, 6, 7]).takeOrdered(6)
    [1, 2, 3, 4, 5, 6]
    >>> sc.parallelize([10, 1, 2, 9, 3, 4, 5, 6, 7], 2).takeOrdered(6, key=lambda x: -x)
    [10, 9, 7, 6, 5, 4]



In [221]:
counts.takeOrdered(15,lambda tupla: -tupla[1])

[('the', 27267),
 ('and', 25340),
 ('i', 19540),
 ('to', 18656),
 ('of', 17301),
 ('a', 14365),
 ('my', 12456),
 ('in', 10660),
 ('you', 10597),
 ('that', 10256),
 ('is', 8681),
 ('with', 7706),
 ('not', 7416),
 ('for', 7297),
 ('his', 6749)]

# Practical

## ETL with airline coupon data

Load the data first: coupon data

In [225]:
!head coupon150720.csv

79062005698500,1,MAA,AUH,9W,9W,56.79,USD,1,H,H,0526,150904,OK,IAF0
79062005698500,2,AUH,CDG,9W,9W,84.34,USD,1,H,H,6120,150905,OK,IAF0
79062005924069,1,CJB,MAA,9W,9W,60.0,USD,1,H,H,2768,150721,OK,IAA0
79065668570385,1,DEL,DXB,9W,9W,160.63,USD,2,S,S,0546,150804,OK,INA0
79065668737021,1,AUH,IXE,9W,9W,152.46,USD,1,V,V,0501,150803,OK,INA0
79062006192650,1,RPR,BOM,9W,9W,68.5,USD,1,K,K,2202,150720,OK,IAE0
79062006192650,2,BOM,RPR,9W,9W,68.5,USD,1,H,H,0377,150721,OK,IAE0
79062005733853,1,DEL,DED,9W,9W,56.16,USD,1,V,V,2839,150801,OK,INA0
79062005836987,1,ATL,LGA,AA,AA,28.3,USD,1,V,V,3237,150903,OK,INB0
79062005836987,2,LGA,EWR,,,0.0,USD,1,,,VOID,,,INA0


In [224]:
coupons = sc.textFile('coupon150720.csv')
coupons.take(5)

['79062005698500,1,MAA,AUH,9W,9W,56.79,USD,1,H,H,0526,150904,OK,IAF0',
 '79062005698500,2,AUH,CDG,9W,9W,84.34,USD,1,H,H,6120,150905,OK,IAF0',
 '79062005924069,1,CJB,MAA,9W,9W,60.0,USD,1,H,H,2768,150721,OK,IAA0',
 '79065668570385,1,DEL,DXB,9W,9W,160.63,USD,2,S,S,0546,150804,OK,INA0',
 '79065668737021,1,AUH,IXE,9W,9W,152.46,USD,1,V,V,0501,150803,OK,INA0']

#### Exercise

Take fields 0, 2, 3, 4, and 6 from each line of coupons

In [228]:
def coupon_from(line):
    split_line = line.split(',')
    field_positions=[0,2,3,4,6]
    fields = tuple(split_line[i] for i in field_positions)
    #fields = (split_line[0],split_line[2],split_line[3],split_line[4],split_line[6])
    return fields
coupon_from('79062005698500,1,MAA,AUH,9W,9W,56.79,USD,1,H,H,0526,150904,OK,IAF0')

('79062005698500', 'MAA', 'AUH', '9W', '56.79')

In [229]:
split_coupons = coupons.map(coupon_from)

In [230]:
split_coupons.take(5)

[('79062005698500', 'MAA', 'AUH', '9W', '56.79'),
 ('79062005698500', 'AUH', 'CDG', '9W', '84.34'),
 ('79062005924069', 'CJB', 'MAA', '9W', '60.0'),
 ('79065668570385', 'DEL', 'DXB', '9W', '160.63'),
 ('79065668737021', 'AUH', 'IXE', '9W', '152.46')]

#### Intermission: Stack traces in Spark

Stack traces in pyspark are quite convoluted because we have many layers of processing. Look for the python stack trace contained within the Py4JJavaError message.

#### Exercise

Keep only the amount. Get average, max, min and std

In [231]:
coupon_values = split_coupons.map(lambda coupon: float(coupon[4]))
coupon_values.take(5)

[56.79, 84.34, 60.0, 160.63, 152.46]

In [232]:
def max_of_two(a,b):
    return a if a>b else b
def min_of_two(a,b):
    return a if a<b else b
max_of_two(3,5)

5

In [233]:
coupon_values.reduce(max_of_two)

6355194.0

In [234]:
coupon_values.reduce(min_of_two)

0.0

In [243]:
sum_sq, sum_, count_ = coupon_values\
       .filter(lambda value: value<1E6)\
       .map(lambda value: (value**2, value, 1))\
       .reduce(lambda t1,t2:(t1[0]+t2[0],t1[1]+t2[1],t1[2]+t2[2]))
sum_sq, sum_,count_

#### All in one go!



In [249]:
sum_sq, sum_, count_, min_, max_= coupon_values\
       .filter(lambda value: value<1E6)\
       .map(lambda value: (value**2, value, 1,value,value))\
       .reduce(lambda t1,t2:(t1[0]+t2[0],
                             t1[1]+t2[1],
                             t1[2]+t2[2],
                             min_of_two(t1[3],t2[3]),
                             max_of_two(t1[4],t2[4])))
print(sum_sq, sum_,count_,min_,max_)
print(sum_/count_)

1598526798129.8743 165766316.49994704 1232659 0.0 637313.52
134.47864859620304


In [246]:
mean

134.47864859620304

#### Exercise

Get stats on ticket amount for all tickets with destination MAD

You will need to extract ticket amounts with destination MAD, and then calculate:

1. Total ticket amounts per origin
2. Top 10 airlines by average amount

#### Part 1

We need to filter coupons with destination Madrid, and after that group on the origin. 

In [250]:
split_coupons.take(5)

[('79062005698500', 'MAA', 'AUH', '9W', '56.79'),
 ('79062005698500', 'AUH', 'CDG', '9W', '84.34'),
 ('79062005924069', 'CJB', 'MAA', '9W', '60.0'),
 ('79065668570385', 'DEL', 'DXB', '9W', '160.63'),
 ('79065668737021', 'AUH', 'IXE', '9W', '152.46')]

In [251]:
mad_coupons = split_coupons.filter(lambda coupon: coupon[2]=='MAD')
mad_coupons.take(10)

[('79062005639642', 'BRU', 'MAD', 'UX', '21.02'),
 ('79065668754871', 'BRU', 'MAD', 'SN', '27.66'),
 ('79065668917696', 'CDG', 'MAD', 'AF', '46.97'),
 ('79062006133090', 'CDG', 'MAD', 'AF', '3.38'),
 ('79062006110497', 'CDG', 'MAD', 'AF', '26.02'),
 ('79062006110505', 'CDG', 'MAD', 'AF', '26.02'),
 ('79062006137072', 'ORY', 'MAD', 'UX', '87.2'),
 ('79062006116586', 'CDG', 'MAD', 'AF', '31.51'),
 ('79062006128336', 'CDG', 'MAD', 'AF', '26.63'),
 ('79062006163025', 'JFK', 'MAD', 'DL', '110.28')]

At this point, we only need the origin, to group on, and the value, to sum

In [252]:
indexed = mad_coupons.map(lambda coupon: (coupon[1], float(coupon[4])))
indexed.take(5)

[('BRU', 21.02), ('BRU', 27.66), ('CDG', 46.97), ('CDG', 3.38), ('CDG', 26.02)]

We are now ready to reduce all amounts per origin.

In [253]:
indexed.reduceByKey(lambda a,b: a +b).takeOrdered(10,key=lambda c: -c[1])

[('CCS', 94528.68),
 ('GRU', 87192.63999999998),
 ('EZE', 81074.63999999997),
 ('BOG', 74644.45000000001),
 ('LHR', 69609.53000000003),
 ('LPA', 60483.92),
 ('MEX', 56316.73000000001),
 ('JFK', 53496.169999999984),
 ('TLV', 53436.220000000016),
 ('TFN', 50034.949999999866)]

#### Part 2

This is very similar, with two differences: we need to group on the airline, and to calculate averages we need to keep track of both the total amount on the coupons and the number of coupons.

In [254]:
coupon_values_by_airline = split_coupons.map(lambda coupon: (coupon[3], (float(coupon[4]),1)))
coupon_values_by_airline.take(5)

[('9W', (56.79, 1)),
 ('9W', (84.34, 1)),
 ('9W', (60.0, 1)),
 ('9W', (160.63, 1)),
 ('9W', (152.46, 1))]

Now we reduce tuples, summing each component of the tuple separately.

In [257]:
totals_by_airline = coupon_values_by_airline\
                .reduceByKey(lambda t1,t2: (t1[0]+t2[0], t1[1]+t2[1]))
totals_by_airline.take(5)

[('9W', (899931.5599999992, 9697)),
 ('AA', (4151996.4599999613, 30565)),
 ('', (478.31000000000006, 18811)),
 ('UA', (5151815.29, 33413)),
 ('UL', (526684.1600000003, 3491))]

This reduceByKey generates an RDD of the form (k, (v1, v2)). We can map it like this:

In [261]:
means_by_airline = totals_by_airline.map(lambda t: (t[0], t[1][0]/t[1][1]))

In [263]:
means_by_airline.take(10)

[('9W', 92.80515210889958),
 ('AA', 135.84153312612338),
 ('', 0.025427143692520335),
 ('UA', 154.1859542692964),
 ('UL', 150.86913778287033),
 ('AC', 180.54509551310488),
 ('WS', 123.3909981343283),
 ('GA', 991.2527671691723),
 ('NH', 239.1886776764052),
 ('PG', 91.59521698113166)]

Or we can use the mapValues method to only process the (v1, v2) part, ignoring the key for processing without discarding it.

In [265]:
means_by_airline = totals_by_airline.mapValues(lambda t: t[0]/t[1])

In [266]:
means_by_airline.take(10)

[('9W', 92.80515210889958),
 ('AA', 135.84153312612338),
 ('', 0.025427143692520335),
 ('UA', 154.1859542692964),
 ('UL', 150.86913778287033),
 ('AC', 180.54509551310488),
 ('WS', 123.3909981343283),
 ('GA', 991.2527671691723),
 ('NH', 239.1886776764052),
 ('PG', 91.59521698113166)]

And we're ready to take our result.

In [267]:
means_by_airline.takeOrdered(10, lambda t: -t[1])

[('S3', 5225.068852459017),
 ('9V', 1488.6739864864867),
 ('GA', 991.2527671691723),
 ('TN', 964.6472580645161),
 ('7F', 668.035),
 ('DT', 579.0006976744188),
 ('B8', 443.295),
 ('V0', 438.6122773044147),
 ('NE', 430.1),
 ('4M', 377.7557471264368)]

#### Stretch: 

Get the totals from first origin without using the coupon number

We'll use groupBy this time, because we need to look at every coupon in a ticket in order to identify first origin.

A ResultIterable is a lazy collection, so we can take it and iterate over it, or turn it into a list to materialize it. This is useful to get a test case to test the function we will be writing.

We are going to compare the set of origins to the set of destinations. The one that is in the first but not in the second should be the first origin.

Now we have have our values, so we are ready to get the totals and sort. Before we reduceByKey, we need to reshape our tuples so the first origin is the key.

### Further resources

https://s3.amazonaws.com/assets.datacamp.com/blog_assets/PySpark_Cheat_Sheet_Python.pdf